In [0]:
## Imports
import sim
import solve
import rulesets
import present
import matplotlib.pyplot as plt
import plot_utils
import display_utils
from display_utils import set

import IPython.display

##############################################################################

# Configuration

## Set simulation parameters below:

## Birth year
year_of_birth = set.this.value

## Retirement goals
age_at_retirement = 65
age_at_death = 90
savings_at_death = 0
retirement_income = set.this.value

## Initial conditions
initial_year = 2019
initial_salary = set.this.value
initial_savings_rrsp = set.this.value
initial_savings_tfsa = set.this.value

## Salary model parameters
# salary_plateau = the most you will ever earn - your salary will plateau when it reaches this figure
# salary_compound_rate = the rate at which your salary compounds per year, until it reaches the plateau. (Aka, your yearly raise.)
salary_plateau = set.this.value
salary_compound_rate = set.this.value

## Spending model parameters
# increase_savings_weight determines how the simulation will try to allot increases in income. If increase_savings_weight = 1, any increase in salary will be entirely put into savings. If increase_savings_weight = 0, any increase in salary will be entirely spent. Intermediate values will distribute between increased savings and increased spending accordingly.
# Note that the total career savings is effectively fixed by the requested retirement income. With that in mind, the effect of a high increase_savings_weight will be to save more 'late' and less 'early,' whereas a low increase_savings_weight will bias the simulation to saving more 'early' and proportionally less 'late.' A matter of personal taste, in other words.
increase_savings_weight = 0.5

# The interest rate earned by savings.
interest_rate = 0.06

## Savings model parameters
# initial_rrsp_allotment = The fraction of savings that will be put into RRSP in the first year of the simulation. The rest will be put into TFSA
# initial_rrsp_allotment = The fraction of savings that will be put into RRSP in the last year before retirement. The rest will be put into TFSA
# The allotment at any year in between will be a linear interpolation between these two values. 
# The simulation will attempt to optimize these values if optimization is enabled.
initial_rrsp_allotment = 0.01
final_rrsp_allotment = 0.2

# During retirement, the simulation will withdraw money from RRSP and TFSA savings to provide the desired retirement income according to a simple heuristic, adjusted by an amount determined by the parameter below. This parameter will be optimized if optimization is enabled.
rrsp_retirement_adjustment = 0.05

# If true, the simulation will try to optimize certain parameters (eg the division between RRSP and TFSA contributions). This may take longer to run!
should_optimize = False

##############################################################################

# The code below sets up and runs the simulation
simulation = sim.Simulation()
simulation.year_of_birth = year_of_birth
simulation.age_at_retirement = age_at_retirement
simulation.age_at_death = age_at_death
simulation.savings_at_death = savings_at_death
simulation.initial_year = initial_year
simulation.initial_salary = initial_salary
simulation.initial_savings_rrsp = initial_savings_rrsp
simulation.initial_savings_tfsa = initial_savings_tfsa

optimize = solve.Optimizing_Solver(solve.binary_solver, should_invert = True)
optimize.is_optimization_disabled = not should_optimize

simulation.set_solver(optimize.solve)

career_rules, retirement_rules = rulesets.hawking(
    salary_compound_rate=salary_compound_rate,
    salary_plateau=salary_plateau,
    increase_savings_weight=increase_savings_weight,
    initial_rrsp_allotment_guess=initial_rrsp_allotment,
    final_rrsp_allotment_guess=final_rrsp_allotment,
    initial_year=simulation.initial_year,
    year_of_retirement=simulation.year_of_retirement,
    year_of_death=simulation.year_of_death,
    retirement_income=retirement_income,
    rrsp_interest_rate=interest_rate,
    tfsa_interest_rate=interest_rate,
    rrsp_retirement_adjustment_guess=rrsp_retirement_adjustment,
    optimize=optimize
)

simulation.set_rules(career_rules)
simulation.set_retirement_rules(retirement_rules)

simulation.run()

presenter = present.Simulation_Presenter(simulation)

##############################################################################

# Text output
if (not simulation.was_solution_found):
    display(display_utils.with_colour("**No solution was found for given inputs! Showing closest outcome.**", "red"))
    display(display_utils.with_colour(f"Message: {simulation.run_message}", "red"))
else:
    display(display_utils.with_colour("Solution found.", "green"))
## Inputs and result
display(display_utils.with_colour(f"<br>", "gray"))
display(display_utils.with_colour(f"Retiring in {simulation.year_of_retirement} at age {simulation.age_at_retirement} with an income of ${retirement_income:,.2f} until age {simulation.age_at_death}", "gray"))
display(display_utils.with_colour(f"Initial after-tax salary: ${presenter.career_net_income_series[0]:,.2f}", "gray"))
display(display_utils.with_colour(f"Initial spending required: ${simulation.required_initial_spending:,.2f}", "gray"))
display(display_utils.with_colour(f"<br>", "gray"))

## Savings table
savings_table = display_utils.table("Year", "Savings per month", "Savings per year", "Projected yearly net income", "Projected yearly pre-tax salary")
YEARS_TO_SHOW_SAVINGS_FOR = 5
for i in range(1, YEARS_TO_SHOW_SAVINGS_FOR +1):
    savings_table.append_row(f"**{presenter.career_years_series[i]}**", 
        f"${presenter.career_total_savings_monthly_series[i]:,.2f}",
        f"${presenter.career_total_savings_series[i]:,.2f}",
        f"${presenter.career_net_income_series[i]:,.2f}",
        f"${presenter.career_salary_series[i]:,.2f}"
    )
display(savings_table.close())
display(display_utils.with_colour(f"<br>", "gray"))

# Graphing

## Salary and spending
fig, ax = plt.subplots()
ax.plot(presenter.career_years_series, presenter.career_salary_series, '-', label="Yearly salary (pre-tax)")
ax.plot(presenter.career_years_series, presenter.career_net_income_series, '-', label="Yearly salary (after tax, + refund)")
ax.plot(presenter.years_series, presenter.spending_series, '-', label="Yearly spending")
ax.axvline(x=presenter.year_of_retirement, color="red", linestyle='--', label="Retirement")
ax.set_ybound(lower=0)
ax.set_xlabel("Year")
ax.yaxis.set_major_formatter(plot_utils.dollar_formatter())
ax.set_title("Salary and spending over time")
ax.legend()

## Absolute savings
fig2, ax2 = plt.subplots()
ax2.stackplot(presenter.years_series, [presenter.rrsp_total_series, presenter.tfsa_total_series], labels=["RRSP", "TFSA"])
ax2.axvline(x=presenter.year_of_retirement, color="red", linestyle='--', label="Retirement")
ax2.yaxis.set_major_formatter(plot_utils.dollar_formatter())
ax2.legend()

## Savings contribution and withdrawal
fig3, ax3 = plt.subplots()
ax3.plot(presenter.career_years_series, presenter.career_rrsp_contribution_series, label="RRSP contributions")
ax3.plot(presenter.career_years_series, presenter.career_tfsa_contribution_series, label="TFSA contributions")
ax3.plot(presenter.retirement_years_series, presenter.retirement_rrsp_withdrawal_series, label="RRSP withdrawals")
ax3.plot(presenter.retirement_years_series, presenter.retirement_tfsa_withdrawal_series, label="TFSA withdrawals")
ax3.axvline(x=presenter.year_of_retirement, color="red", linestyle='--', label="Retirement")
ax3.axhline(y=0, color="black")
ax3.yaxis.set_major_formatter(plot_utils.dollar_formatter())
ax3.legend()

plt.show()

if (should_optimize):
    display("Optimized values:")
    display(list(optimize.get_all_optimized_values()))
    display("Un-optimized initial output:")
    display(optimize.initial_output)
    display("Un-optimized values:")
    display(list(optimize.get_all_initial_solution_values()))

In [11]:
# Re-import modified dependencies
import importlib
import present
import sim
import rulesets
import spending_rules
import savings_rules
import solve
import display_utils

importlib.reload(present)
importlib.reload(sim)
importlib.reload(rulesets)
importlib.reload(spending_rules)
importlib.reload(solve)
importlib.reload(display_utils)
importlib.reload(savings_rules)

display("Reimported modules")

In [ ]:
#

